In [1]:
#функция вычисляющая методом дихотомии точку пересечения с Ox
def dichotomy(fun, a, b, eps=1e-5):
    if fun(x=a)*fun(x=b) > 0:
        return None
    median = (a+b)/2.0
    if abs(fun(x=median).n()) < eps:
        return median
    if fun(x=median)*fun(x=a) < 0:
        return dichotomy(fun, a, median)
    else:
        return dichotomy(fun, median, b)

In [2]:

#функция вычисляющая методом Ньютона точку пересечения с Ox
def newton(fun, a, b, eps=1e-5):
    x1 = a
    x2 = (a + b) / 2
    df = fun.derivative()
    while abs(x2 - x1) >= eps:
        x1 = x2
        x2 = x1 - fun(x=x1)/df(x=x1)
    if abs(fun(x=x1)) <= 1e-3:
        return x1
    else:
        return None

In [3]:
#значения которые будут доступны для интерактивного взаимодействия
a = (0, 2, 0.5)
b = (0, 2, 0.5)
step = (1, 2, 1)

In [4]:
#декоратор для интерактивного взаимодействия, включающий вызов функций из пред. шагов
@interact
def funcs_interact(a=a, b=b, step=step):
    #некоторые нужные константы
    xmin = -2
    xmax = 8
    ymin = -5
    ymax = 7
    eps = 1e-5
    
    #сама функция
    f(x) = x^3 - 9*x^2 + 23*x - 15
    
    #значения корней полученных методом дихотомии
    dich = [dichotomy(f, a, b, eps)]
    dich.append(dichotomy(f, a+step, b+step, eps))
    dich.append(dichotomy(f, a+2*step, b+2*step, eps))
    
    #значения корней полученных методом Ньютона
    newt = [newton(f, a, b, eps)]
    newt.append(newton(f, a+step, b+step, eps))
    newt.append(newton(f, a+2*step, b+2*step, eps))
    
    
    #первые три графика методом дихотомии
    gr = [plot(f, (x, xmin, xmax), ymin=ymin, ymax=ymax, aspect_ratio=1, figsize=[3,2], color='green')]*3
    gr.extend([plot(f, (x, xmin, xmax), ymin=ymin, ymax=ymax, aspect_ratio=1, figsize=[3,2], color='blue')]*3)
    for i in range(len(gr)):
        if i <= 2 and dich[i] is not None:
            gr[i] += point((float(dich[i]), 0), color='red', aspect_ratio=1, size=30)
        if i > 2 and newt[i-3] is not None:
            gr[i] += point((float(newt[i-3]), 0), color='red', aspect_ratio=1, size=30)
    
    #вывод массива графиков на один холст
    graphics_array([[gr[0], gr[1], gr[2]], [gr[3], gr[4], gr[5]]]).show(frame=True)
       #Печать на экран значений корней
    my_str = "Корни полученные методом дихотомии: "
    for d in dich:
        if d is not None:
            my_str += "{0:.1}".format(d) + " "
    my_str += "\nКорни полученные методом Ньютона: "
    for n in newt:
        if n is not None:
            my_str += str(round(float(n))) + " "
    print(my_str)

Interactive function <function funcs_interact at 0x6fff0e6e4710> with 3 widgets
  a: FloatSlider(value=1.0, de…

In [5]:

#результат полученный функцией solve
f(x) = x^3 - 9*x^2 + 23*x - 15
solve([f(x) == 0], x)

[x == 1, x == 5, x == 3]

In [6]:
#полученный НОД при помощи стандартной функции gcd
R, (x, y) = PolynomialRing(QQ,'x, y').objgens()
f = x^4 - 4*x^3 + 7*x^2 - 5*x + 1
g = x^5 - x^4 - x^3 + x^2 + 5*x - 5
res_gcd = gcd(f, g)
print("НОД =", str(res_gcd))

НОД = x - 1


In [7]:
#функция вычисления коэффициентов тождества Безу и gcd(a,b)
def bezout(a, b):
    x, xx, y, yy = 1, 0, 0, 1
    while b:
        q, r = a // b, a % b
        a = b
        b = r
        x, xx = xx, x - xx*q
        y, yy = yy, y - yy*q
    return (x, y, a)

In [8]:
#получаем результат в виде tuple трех элементов u, v, gcd(a,b)
bez = bezout(f, g)

In [9]:
#полученный gcd(a, b) - левая часть тождества
print("НОД =", f*bez[0]+bez[1]*g)

НОД = 1936/441*x - 1936/441


In [10]:
#полученный gcd(a,b) - правая часть тождества
print("НОД =", bez[2])

НОД = 1936/441*x - 1936/441


In [11]:
#проверка тождества Безу f*u + v*g = gcd(f, g)
if (f*bez[0]+bez[1]*g) - bez[2] == 0:
    print("Получено верное тождество")
else:
    print("Получено неверное тождество")

Получено верное тождество


In [12]:
#необходимые константы
xmin, xmax = -10, 10
ymin, ymax = -10, 10
zmin, zmax = -10, 10

In [13]:
#задаем нашу функцию
var("x y z")
f(x, y, z) = 7*x^2 + 3*y^2 + 3*z^2 +  8*x*y + 8*x*z + 6*y*z + 6*x + y + 7

In [14]:
#построим график исходной фигуры заданной неявно
implicit_plot3d(f(x=x, y=y, z=z), (x, xmin, xmax), (y, ymin, ymax), (z, zmin, zmax))

Graphics3d Object

In [15]:
#функция приведения уравнения поверхности второго порядка к каноническому виду, работает для любой функции у которой все l
#разные и матрица квадратичной формы не диагональная
def kanonic_coeffs(fun):
    try:
        var("l l1 l2 l3")
        lvcts = []
        svcts = []
        tmp_fun = fun
        a = vector(RR, 3)
        tmp_vct = vector(RR, 9)
        var_combs_tmp = (x^2, y^2, z^2, x*y, x*z, y*z)
        var_combs_0 = (x^2, x*y, x*z, x*y, y^2, y*z, x*z, y*z, z^2)
        var_combs_1 = (x, y, z)
        for i, var_comb in enumerate(var_combs_0):
            if i == 0 or i == 4 or i == 8:
                tmp_vct[i] = fun.coefficient(var_comb)
            else:
                tmp_vct[i] = fun.coefficient(var_comb) / 2
        for var_comb in var_combs_tmp:
            tmp_fun -= fun.coefficient(var_comb)*var_comb
        for i, var_comb in enumerate(var_combs_1):
            a[i] = tmp_fun.coefficient(var_comb) / 2
            tmp_fun -= tmp_fun.coefficient(var_comb)*var_comb
        a0 = tmp_fun.n()    
        A = matrix(SR, 3, tmp_vct)
        L = matrix(SR, 3, 3, var('l'))
        E = matrix(SR, 3, 3, 1)
        lvct = vector([l1, l2, l3])
        lambdas = solve([(A-L).determinant() == 0], l)
        for i, el in enumerate(lambdas):
            nums = []
            lhs = (A-el.rhs()*E)*lvct
            res = solve([lhs[0] == 0, lhs[1] == 0, lhs[2] == 0], l1, l2, l3)[0]
            for i in range(len(res)):
                if len(res[i].rhs().variables()) == 0:
                    nums.append(res[i].rhs())
                                    else:
                    nums.append(res[i].rhs()(1))
            lvcts.append(vector(nums))
        for el in lvcts:
            norm_lvct = (el / sqrt((el*el).n())).n()
            svcts.append(norm_lvct)
        ST = matrix(RR, 3)
        for i in range(len(svcts)):
            ST[i] = svcts[i]
        a_ = ST*a
        for i in range(len(lambdas)):
            lambdas[i] = lambdas[i].rhs().n()
        return (lambdas, a_, a0)
    except:
        print("Something gone wrong\n")
        return (None, None, None)


IndentationError: unexpected indent (<ipython-input-15-920c2feab922>, line 37)

In [ ]:
#выведем полученные коэффициенты, они соответсвуют тем, которые можно получить посчитав вручную
print("Lambdas:", tuple(lambdas))
print("a:", a)
print("a0:", a0)

In [ ]:
#создаем функцию с нужными коэффициентами
var("kx ky kz")
kanonic_func(kx, ky, kz) = lambdas[0]*kx^2 + lambdas[1]*ky^2 + lambdas[2]*kz^2 + 2*a[0]*kx + 2*a[1]*ky + 2*a[2]*kz + a0

In [ ]:
#выведем полученную формулу
show(kanonic_func(kx, ky, kz))

In [ ]:
#построим график преобразованной фигуры 
implicit_plot3d(kanonic_func(kx=kx, ky=ky, kz=kz), (kx, xmin, xmax), (ky, ymin, ymax), (kz, zmin, zmax))